In [4]:
import agate

In [113]:
tester_15 = agate.TypeTester(limit=100,force={
    'NRC_DESC': agate.Text(),
    'NRC_CODE': agate.Number(),
        'BEDSCODE': agate.Text(),
        'L1_COUNT': agate.Text(),
        'L1_PCT': agate.Text(),
        'L2_COUNT': agate.Text(),
        'L2_PCT': agate.Text(),
        'L3_COUNT': agate.Text(),
        'L3_PCT': agate.Text(),
        'L4_COUNT': agate.Text(),
        'L4_PCT': agate.Text(),
        'L2-L4_PCT': agate.Text(),
        'L3-L4_PCT': agate.Text(),
        'MEAN_SCALE_SCORE': agate.Text(),
        'COUNTY_DESC': agate.Text(),
        'TOTAL_TESTED': agate.Text(),
        'COUNTY_CODE': agate.Text(),
        'Sum_Of_SCALE_SCORE': agate.Text()
})
tester_16 = agate.TypeTester(limit=100,force={
    'NRC_DESC': agate.Text(),
    'NRC_CODE': agate.Number(),
        'BEDSCODE': agate.Text(),
        'L1_COUNT': agate.Text(),
        'L1_PCT': agate.Text(),
        'L2_COUNT': agate.Text(),
        'L2_PCT': agate.Text(),
        'L3_COUNT': agate.Text(),
        'L3_PCT': agate.Text(),
        'L4_COUNT': agate.Text(),
        'L4_PCT': agate.Text(),
        'L2-L4_PCT': agate.Text(),
        'L3-L4_PCT': agate.Text(),
        'MEAN_SCALE_SCORE': agate.Text(),
        'COUNTY_DESC': agate.Text(),
        'TOTAL_TESTED': agate.Text(),
        'COUNTY_CODE': agate.Text()
})

In [114]:
def get_percent_dictionary(file_name, tester):
    schools = agate.Table.from_csv(file_name, column_types=tester)
    erie_niagara = schools.where(lambda row: row['COUNTY_DESC'] in ['ERIE', 'NIAGARA'])
    schools_total = erie_niagara.where(lambda row: row['SUBGROUP_NAME'] in ['All Students'])
    non_county = schools_total.where(lambda row: row['NAME'] not in ['ERIE COUNTY', 'NIAGARA COUNTY'])
    print(len(erie_niagara.rows))
    print(len(non_county.rows))
    beds_code = {}
    for row in non_county.rows:
        if 'SCHOOL DISTRICT' in row['NAME']:
            beds_district = row['BEDSCODE'][:6]
            beds_code[str(beds_district)] = row['NAME']
    def calculate_counts(row):
        try:
            l3_counts = int(row['L3_COUNT'])
        except ValueError:
            l3_counts = 0
        try:
            l4_counts = int(row['L4_COUNT'])
        except ValueError:
            l4_counts = 0
        profficent = l3_counts + l4_counts
        return profficent
    erie_calcs = non_county.compute([
        ('pass_count', agate.Formula(agate.Number(), lambda row: calculate_counts(row)))
    ])
    percent_calculates = {}
    for row in erie_calcs.rows:
        if row['NAME'] == 'NIAGARA CHARTER SCHOOL':
            district = 'NIAGARA-WHEATFIELD CENTRAL SCHOOL DISTRICT'
        else:
            beds_district = row['BEDSCODE'][:6]
            district = beds_code[beds_district]
        percent_calculates[row['BEDSCODE']] = {'name': row['NAME'], 'math': {'totals': [], 'profficent': [], 'classes': []}, 'ela': {'totals': [], 'profficent': [], 'classes': []}, 'district': district, 'county': row['COUNTY_DESC'] }
    for row in erie_calcs.rows:
        if row['ITEM_SUBJECT_AREA'] == 'ELA':
            data_dict = percent_calculates[row['BEDSCODE']]['ela']
        else:
            data_dict = percent_calculates[row['BEDSCODE']]['math']
        try:
            data_dict['totals'].append(int(row['TOTAL_TESTED']))
        except ValueError:
            data_dict['totals'].append(0)
        try:
            data_dict['classes'].append(row['ITEM_DESC'])
        except ValueError:
            data_dict['classes'].append(0)
        try:
            data_dict['profficent'].append(int(row['pass_count']))
        except ValueError:
            data_dict['profficent'].append(0)
    for school, values in percent_calculates.items():
        total_sum = sum(values['ela']['totals'])
        profficient_sum = sum(values['ela']['profficent'])
        try:
            correct_value = ((profficient_sum / total_sum) * 100)
            percent_calculates[school]['ela']['total_percent'] = float("{0:.1f}".format(correct_value))
        except ZeroDivisionError:
            percent_calculates[school]['ela']['total_percent'] = '-'
        total_math_sum = sum(values['math']['totals'])
        profficient_math_sum = sum(values['math']['profficent'])
        try:
            correct_math_value = ((profficient_math_sum / total_math_sum) * 100)
            percent_calculates[school]['math']['total_percent'] = float("{0:.1f}".format(correct_math_value))
        except ZeroDivisionError:
            percent_calculates[school]['math']['total_percent'] = '-'
    return percent_calculates

In [116]:
year_16 = get_percent_dictionary('3-8_ELA_MATH_2016.csv', tester_16)


/Users/lheinle/anaconda/envs/py35/lib/python3.5/site-packages/agate/table/__init__.py:104: DuplicateColumnWarning: Column name "" already exists in Table. Column will be renamed to "_2".
  warn_duplicate_column(new_column_name, final_column_name)


25544
1837


In [117]:
year_17 = get_percent_dictionary('3-8_ELA_MATH_2017.csv', tester_16)

25452
1815


In [118]:
year_15 = get_percent_dictionary('3-8_ELA_MATH_2015.csv',tester_15)

25716
1852


In [120]:
print(year_16['400400010001'])
print(year_15['400400010001'])
print(year_17['400400010001'])

{'name': 'ANNA MERRITT ELEMENTARY SCHOOL', 'ela': {'classes': ['Grade 3 ELA', 'Grade 4 ELA'], 'total_percent': 25.2, 'profficent': [22, 11], 'totals': [69, 62]}, 'county': 'NIAGARA', 'district': 'LOCKPORT CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 3 Math', 'Grade 4 Math'], 'total_percent': 41.3, 'profficent': [29, 21], 'totals': [67, 54]}}
{'name': 'ANNA MERRITT ELEMENTARY SCHOOL', 'ela': {'classes': ['Grade 3 ELA', 'Grade 4 ELA'], 'total_percent': 17.8, 'profficent': [11, 12], 'totals': [60, 69]}, 'county': 'NIAGARA', 'district': 'LOCKPORT CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 3 Math', 'Grade 4 Math'], 'total_percent': 37.0, 'profficent': [15, 29], 'totals': [57, 62]}}
{'name': 'ANNA MERRITT ELEMENTARY SCHOOL', 'ela': {'classes': ['Grade 3 ELA', 'Grade 4 ELA'], 'total_percent': 22.9, 'profficent': [11, 16], 'totals': [53, 65]}, 'county': 'NIAGARA', 'district': 'LOCKPORT CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 3 Math', 'Grade 4 Math'], 'total_percent': 3

In [191]:
erie_list = {}
niagara_list = {}
for key, value in year_16.items():
    if 'SCHOOL DISTRICT' in value['name']:
        if value['county'] == 'ERIE':
            erie_list[key] = {'name': value['name'], 'schools': {}}
        else:
            niagara_list[key] = {'name': value['name'], 'schools': {}}
for distict, value in erie_list.items():
    for school, info in year_16.items():
        if value['name'] == info['district'] and 'SCHOOL DISTRICT' not in info['name']:
            erie_list[distict]['schools'][school] = info['name']
for distict, value in niagara_list.items():
    for school, info in year_16.items():
        if value['name'] == info['district'] and 'SCHOOL DISTRICT' not in info['name']:
            niagara_list[distict]['schools'][school] = info['name']

In [214]:
#LAFAYETTE HIGH SCHOOL -> NEWCOMER ACADEMY AT LAFAYETTE
print(year_15['140600010107'])
print(year_16['140600010107'])
print(year_17['140600010309'])

{'name': 'LAFAYETTE HIGH SCHOOL', 'ela': {'classes': ['Grade 7 ELA', 'Grade 8 ELA'], 'total_percent': 3.8, 'profficent': [4, 2], 'totals': [66, 91]}, 'county': 'ERIE', 'district': 'BUFFALO CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 7 Math', 'Grade 8 Math'], 'total_percent': 4.1, 'profficent': [5, 2], 'totals': [73, 99]}}
{'name': 'LAFAYETTE HIGH SCHOOL', 'ela': {'classes': ['Grade 8 ELA'], 'total_percent': 9.5, 'profficent': [6], 'totals': [63]}, 'county': 'ERIE', 'district': 'BUFFALO CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 8 Math'], 'total_percent': 6.6, 'profficent': [4], 'totals': [61]}}
{'name': 'NEWCOMER ACADEMY AT LAFAYETTE', 'ela': {'classes': ['Grade 7 ELA', 'Grade 8 ELA'], 'total_percent': '-', 'profficent': [0, 0], 'totals': [0, 0]}, 'county': 'ERIE', 'district': 'BUFFALO CITY SCHOOL DISTRICT', 'math': {'classes': ['Grade 7 Math', 'Grade 8 Math'], 'total_percent': 1.9, 'profficent': [1, 0], 'totals': [14, 38]}}


In [209]:
def percent_change(new,old):
    try:
        calculate = (new-old)/old
        percent = calculate * 100
        one_decimal = float("{0:.1f}".format(percent))
        return one_decimal
    except (ZeroDivisionError,TypeError):
        return 0

In [215]:
import csv
import operator
erie_districts = 0
niagara_districts = 0
with open('print_out.tsv', 'w') as out_put:
    writer = csv.writer(out_put, delimiter='\t')
    #writer.writerow(['school','ELA 2015', 'ELA 2016', 'ELA 2017', 'Math 2015', 'Math 2016', 'Math 2017', 'district'])
    writer.writerow(['school','ELA 2015', 'ELA 2016', 'ELA 2017', 'Math 2015', 'Math 2016', 'Math 2017', 'pc_ela_15_17', 'pc_ela_16_17', 'pc_math_15_17', 'pc_math_16_17'])
    sorted_erie = sorted(erie_list, key= lambda district: erie_list[district]['name'])
    for district in sorted_erie:
        erie_districts += 1
        #print(erie_list[district])
        print('*** starting district {0}'.format(erie_list[district]['name']))
        pc_ela_15_17 = percent_change(year_17[district]['ela']['total_percent'],year_15[district]['ela']['total_percent'])
        pc_ela_16_17 = percent_change(year_17[district]['ela']['total_percent'],year_16[district]['ela']['total_percent'])
        pc_math_15_17 = percent_change(year_17[district]['math']['total_percent'],year_15[district]['math']['total_percent'])
        pc_math_16_17 = percent_change(year_17[district]['math']['total_percent'],year_16[district]['math']['total_percent'])
        district_info = [erie_list[district]['name'], year_15[district]['ela']['total_percent'],year_16[district]['ela']['total_percent'],year_17[district]['ela']['total_percent'],year_15[district]['math']['total_percent'],year_16[district]['math']['total_percent'],year_17[district]['math']['total_percent'], pc_ela_15_17, pc_ela_16_17, pc_math_15_17, pc_math_16_17]
        #district_info = [erie_list[district]['name'], year_15[district]['ela']['total_percent'],year_16[district]['ela']['total_percent'],year_17[district]['ela']['total_percent'],year_15[district]['math']['total_percent'],year_16[district]['math']['total_percent'],year_17[district]['math']['total_percent'],year_15[district]['district']]
        writer.writerow(district_info)
        schools_info = erie_list[district]['schools']
        #print(schools_info)
        for school in schools_info.keys():
            try:
                #school_info = [schools_info[school], year_15[school]['ela']['total_percent'],year_16[school]['ela']['total_percent'],year_17[school]['ela']['total_percent'],year_15[school]['math']['total_percent'],year_16[school]['math']['total_percent'],year_17[school]['math']['total_percent'],year_15[school]['district']]
                pc_school_ela_15_17 = percent_change(year_17[school]['ela']['total_percent'],year_15[school]['ela']['total_percent'])
                pc_school_ela_16_17 = percent_change(year_17[school]['ela']['total_percent'],year_16[school]['ela']['total_percent'])
                pc_school_math_15_17 = percent_change(year_17[school]['math']['total_percent'],year_15[school]['math']['total_percent'])
                pc_school_math_16_17 = percent_change(year_17[school]['math']['total_percent'],year_16[school]['math']['total_percent'])
                school_info = [schools_info[school], year_15[school]['ela']['total_percent'],year_16[school]['ela']['total_percent'],year_17[school]['ela']['total_percent'],year_15[school]['math']['total_percent'],year_16[school]['math']['total_percent'],year_17[school]['math']['total_percent'], pc_school_ela_15_17, pc_school_ela_16_17, pc_school_math_15_17, pc_school_math_16_17]
                writer.writerow(school_info)
                print('writing the followin {0}'.format(schools_info[school]))
            except KeyError:
                print(schools_info[school])
                print(school)
    sorted_niagara = sorted(niagara_list, key= lambda district: niagara_list[district]['name'])
    for district in sorted_niagara:
        niagara_districts += 1
        print(niagara_list[district])
        print('*** starting district {0}'.format(niagara_list[district]['name']))
        pc_ela_15_17 = percent_change(year_17[district]['ela']['total_percent'],year_15[district]['ela']['total_percent'])
        pc_ela_16_17 = percent_change(year_17[district]['ela']['total_percent'],year_16[district]['ela']['total_percent'])
        pc_math_15_17 = percent_change(year_17[district]['math']['total_percent'],year_15[district]['math']['total_percent'])
        pc_math_16_17 = percent_change(year_17[district]['math']['total_percent'],year_16[district]['math']['total_percent'])
        district_info = [niagara_list[district]['name'], year_15[district]['ela']['total_percent'],year_16[district]['ela']['total_percent'],year_17[district]['ela']['total_percent'],year_15[district]['math']['total_percent'],year_16[district]['math']['total_percent'],year_17[district]['math']['total_percent'], pc_ela_15_17, pc_ela_16_17, pc_math_15_17, pc_math_16_17]
        writer.writerow(district_info)
        schools_info = niagara_list[district]['schools']
        #print(schools_info)
        for school in schools_info.keys():
            try:
                print('writing the followin {0}'.format(schools_info[school]))
                pc_school_ela_15_17 = percent_change(year_17[school]['ela']['total_percent'],year_15[school]['ela']['total_percent'])
                pc_school_ela_16_17 = percent_change(year_17[school]['ela']['total_percent'],year_16[school]['ela']['total_percent'])
                pc_school_math_15_17 = percent_change(year_17[school]['math']['total_percent'],year_15[school]['math']['total_percent'])
                pc_school_math_16_17 = percent_change(year_17[school]['math']['total_percent'],year_16[school]['math']['total_percent'])
                school_info = [schools_info[school], year_15[school]['ela']['total_percent'],year_16[school]['ela']['total_percent'],year_17[school]['ela']['total_percent'],year_15[school]['math']['total_percent'],year_16[school]['math']['total_percent'],year_17[school]['math']['total_percent'], pc_school_ela_15_17, pc_school_ela_16_17, pc_school_math_15_17, pc_school_math_16_17]
                writer.writerow(school_info)
            except KeyError:
                print(schools_info[school])
                print(school)
print('{0} erie districts and {1} niagara districts'.format(erie_districts,niagara_districts))

*** starting district AKRON CENTRAL SCHOOL DISTRICT
writing the followin AKRON MIDDLE SCHOOL
writing the followin AKRON ELEMENTARY SCHOOL
*** starting district ALDEN CENTRAL SCHOOL DISTRICT
writing the followin ALDEN MIDDLE SCHOOL
ALDEN PRIMARY AT TOWNLINE
140101060003
*** starting district AMHERST CENTRAL SCHOOL DISTRICT
writing the followin AMHERST MIDDLE SCHOOL
writing the followin SMALLWOOD DRIVE SCHOOL
writing the followin WINDERMERE BLVD SCHOOL
*** starting district BUFFALO CITY SCHOOL DISTRICT
writing the followin DISCOVERY SCHOOL
writing the followin COMMUNITY SCHOOL #53 AT #4
writing the followin BUILD ACADEMY
writing the followin SOUTH BUFFALO CHARTER SCHOOL
writing the followin SOUTHSIDE ELEMENTARY SCHOOL
writing the followin DR GEORGE BLACKMAN ECC
writing the followin WEST BUFFALO CHARTER SCHOOL
writing the followin BILINGUAL CENTER
writing the followin PS 69 HOUGHTON ACADEMY
writing the followin DR ANTONIA PANTOJA COMMUNITY SCHOOL OF ACADEMIC EXCELLENCE AT #18
writing the 

In [ ]:
140600